In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

### Carregar e visualizar os dados

In [2]:
# Carregar os dados
df = pd.read_csv('./datasets/dataset_colesterol.csv')

In [3]:
# Visualizar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     996 non-null    object 
 2   Fumante             997 non-null    object 
 3   Nível de Atividade  996 non-null    object 
 4   Idade               997 non-null    float64
 5   Peso                997 non-null    float64
 6   Altura              997 non-null    float64
 7   Colesterol          1000 non-null   float64
dtypes: float64(4), int64(1), object(3)
memory usage: 62.6+ KB


In [4]:
# Visualizar Dataframe
df.head(5)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33.0,85.1,186.0,199.63
1,2,A,Não,Moderado,68.0,105.0,184.0,236.98
2,3,O,Não,Alto,25.0,64.8,180.0,161.79
3,4,A,Não,Alto,43.0,120.2,167.0,336.24
4,5,AB,Não,Baixo,79.0,88.5,175.0,226.23


In [5]:
# Ajustar DataFrame

df.drop('Id', axis=1, inplace=True)
df.dropna(inplace=True, axis=0)

# Aplicar OneHotEncoding nas variáveis categóricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [6]:
# DataFrame atualizado
df.head(5)

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33.0,85.1,186.0,199.63,0,0,1,0,0,1,0,1,0
1,68.0,105.0,184.0,236.98,1,0,0,0,1,0,0,0,1
2,25.0,64.8,180.0,161.79,0,0,0,1,1,0,1,0,0
3,43.0,120.2,167.0,336.24,1,0,0,0,1,0,1,0,0
4,79.0,88.5,175.0,226.23,0,1,0,0,1,0,0,1,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Idade                        981 non-null    float64
 1   Peso                         981 non-null    float64
 2   Altura                       981 non-null    float64
 3   Colesterol                   981 non-null    float64
 4   Grupo Sanguíneo_A            981 non-null    uint8  
 5   Grupo Sanguíneo_AB           981 non-null    uint8  
 6   Grupo Sanguíneo_B            981 non-null    uint8  
 7   Grupo Sanguíneo_O            981 non-null    uint8  
 8   Fumante_Não                  981 non-null    uint8  
 9   Fumante_Sim                  981 non-null    uint8  
 10  Nível de Atividade_Alto      981 non-null    uint8  
 11  Nível de Atividade_Baixo     981 non-null    uint8  
 12  Nível de Atividade_Moderado  981 non-null    uint8  
dtypes: float64(4), uint8

### Treinar modelo de regressão linear múltipla com RFE

In [8]:
# Separar X e y
X = df.drop('Colesterol', axis=1)
y=df['Colesterol']

In [9]:
# Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=51)

In [10]:
# Treinar o modelo com RFE
# RFE (Recursive Features Elimination)
# Uso um estimador e define uma quantidade de features (dois hiperparâmetros)
# O RFE faz iterações iniciando com todas as features e eliminando a cada iteração até atingir a quantidade definida
# Elimina características/features menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=6)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=6)

In [11]:
# Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto'],
      dtype='object')

In [12]:
# Ranking de Features
def mostrar_ranking(metodo_fs, X_train):
  
  # Obter o Ranking de Features
  ranking = rfe_method.ranking_

  # Obter os nomes das features
  nomes_features = X_train.columns.to_list()

  # Criar um Dataframe com os rankings e os nomes das features
  df_ranking = pd.DataFrame({'Features': nomes_features, 'Ranking': ranking})

  # Ordenar o DataFrame pelo Ranking
  df_ranking = df_ranking.sort_values(by='Ranking')

  # Exibir Ranking
  print(df_ranking)

In [13]:
# Ranking de Features do RFE Regressão
mostrar_ranking(rfe_method, X_train)

                       Features  Ranking
1                          Peso        1
2                        Altura        1
4            Grupo Sanguíneo_AB        1
7                   Fumante_Não        1
8                   Fumante_Sim        1
9       Nível de Atividade_Alto        1
10     Nível de Atividade_Baixo        2
3             Grupo Sanguíneo_A        3
5             Grupo Sanguíneo_B        4
6             Grupo Sanguíneo_O        5
11  Nível de Atividade_Moderado        6
0                         Idade        7


In [14]:
# Função para avaliar performance
def performance_regressao(modelo, X_test, y_test):
  
  # Faz a predição com o modelo no conjunto de testes
  y_pred = modelo.predict(X_test)

  # Avaliar desempenho
  return mean_squared_error(y_test, y_pred, squared=False)

In [15]:
# Performance Regressão com RFE
performance_regressao(rfe_method, X_test, y_test)

/home/s026703811/.local/share/virtualenvs/AI_development-tfqJyHn1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.99086382164199

### Treinar modelo sem RFE

In [16]:
# Treinar modelo sem RFE
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [17]:
# Performance Regressão sem RFE
performance_regressao(model_reg, X_test, y_test)

/home/s026703811/.local/share/virtualenvs/AI_development-tfqJyHn1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.994321619811462

### Treinar Modelo de Regressão Linear com RFECV

In [18]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [19]:
performance_regressao(rfe_method_cv, X_test, y_test)

/home/s026703811/.local/share/virtualenvs/AI_development-tfqJyHn1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.999842607032079

In [20]:
# Quais features foram selecionadas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Fumante_Não', 'Fumante_Sim', 'Nível de Atividade_Alto',
       'Nível de Atividade_Baixo'],
      dtype='object')

In [21]:
# Quantas Features foram selecionadas
rfe_method_cv.n_features_

8

### Treinar modelo de regressão com SelectFromModel

In [22]:
sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [23]:
# Quais features foram selecionadas
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Fumante_Sim'], dtype='object')

In [24]:
# Treinar modelo com as features selecionadas
X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)

LinearRegression()

In [25]:
# Performance do Modelo com SelectFromModel
performance_regressao(model_reg, X_test_ajustado_reg, y_test)

/home/s026703811/.local/share/virtualenvs/AI_development-tfqJyHn1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.985747977687915